**ALL THE PATHS REQUIRED**

In [1]:
Workspace_path = 'Tensorflow/workspace'
Scripts_path = 'Tensorflow/scripts'
ApiModel_path = 'Tensorflow/models'
Annotation_path = Workspace_path+'/annotations'
Img_path = Workspace_path+'/images'
Model_path = Workspace_path+'/models'
Pretrained_Model_path = Workspace_path+'/pre-trained-models'
Config_path = Model_path+'/ssd_mobnet/pipeline.config'
Checkpoint_path = Model_path+'/ssd_mobnet/'
from matplotlib import pyplot as pl

**SIGN LABELS**

In [11]:
labels = [{'name':'help', 'id':1}, {'name':'more', 'id':2},{'name':'stop', 'id':3},{'name':'want', 'id':4},{'name':'yes', 'id':5}]

with open(Annotation_path + 'sign_label.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

**TF GENERATION**

In [13]:
!python {Scripts_path + '/generate_tfrecord.py'} -x {Img_path + '/train'} -l {Annotation_path + 'sign_label.pbtxt'} -o {Annotation_path + '/train.record'}
!python {Scripts_path + '/generate_tfrecord.py'} -x{Img_path + '/test'} -l {Annotation_path + 'sign_label.pbtxt'} -o {Annotation_path + '/test.record'}

Traceback (most recent call last):
  File "D:\Sign_Language_Detection\Tensorflow\scripts\generate_tfrecord.py", line 29, in <module>
    from object_detection.utils import dataset_util, label_map_util
  File "C:\Users\user\anaconda3\envs\tensorflow\lib\site-packages\object_detection\utils\label_map_util.py", line 29, in <module>
    from object_detection.protos import string_int_label_map_pb2
  File "C:\Users\user\anaconda3\envs\tensorflow\lib\site-packages\object_detection\protos\string_int_label_map_pb2.py", line 5, in <module>
    from google.protobuf.internal import builder as _builder
ImportError: cannot import name 'builder' from 'google.protobuf.internal' (C:\Users\user\anaconda3\envs\tensorflow\lib\site-packages\google\protobuf\internal\__init__.py)
Traceback (most recent call last):
  File "D:\Sign_Language_Detection\Tensorflow\scripts\generate_tfrecord.py", line 29, in <module>
    from object_detection.utils import dataset_util, label_map_util
  File "C:\Users\user\anaconda3

**DOWNLOAD REQUIRED TENSORFLOW MODELS**

In [14]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


**COPYING PIPELINE CONFIG TO TRAIN FOLDER**

In [16]:
Custom_model_name = 'new_ssd_mobnet' 

In [17]:
!mkdir {'Tensorflow\workspace\models\\'+Custom_model_name}
!copy {Pretrained_Model_path+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {Model_path+'/'+Custom_model_name}

<string>:1: SyntaxWarning: invalid escape sequence '\w'
A subdirectory or file Tensorflow\workspace\models\new_ssd_mobnet already exists.


The syntax of the command is incorrect.


**TRANSFER LEARNING**

In [9]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

ModuleNotFoundError: No module named 'object_detection'

In [24]:
Config_path = Model_path+'/'+Custom_model_name+'/pipeline.config'

In [25]:
config = config_util.get_configs_from_pipeline_file(Config_path)

NameError: name 'config_util' is not defined

In [26]:
config

Available objects for config:
    AliasManager
    DisplayFormatter
    HistoryManager
    IPCompleter
    IPKernelApp
    LoggingMagics
    MagicsManager
    OSMagics
    PrefilterManager
    ScriptMagics
    StoreMagics
    ZMQInteractiveShell


In [27]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(Config_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

NameError: name 'pipeline_pb2' is not defined

In [53]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = Pretrained_Model_path+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.sign_label_path= Annotation_path + '/sign_label.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [Annotation_path + '/train.record']
pipeline_config.eval_input_reader[0].sign_label_path = Annotation_path + '/sign_label.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [Annotation_path + '/test.record']

In [54]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(Config_path, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

**MODEL TRAINING**

In [58]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_Config_path={}/{}/pipeline.config --num_train_steps=10000""".format(ApiModel_path, Model_path,Custom_model_name,Model_path,Custom_model_name))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


**MODEL LOADING**

In [2]:
import os
from object_detection.utils import sign_label_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [9]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(Config_path)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(Checkpoint_path, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# REAL TIME DETECTION

In [10]:
import cv2 
import numpy as np

In [11]:
category_index = sign_label_util.create_category_index_from_labelmap(Annotation_path+'/sign_label.pbtxt')

In [105]:
cap.release()

In [12]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [42]:
detections = detect_fn(input_tensor)